In [742]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [743]:
# Adatok
height = 1000
base = 1000
elemSize = 5
traction = 1

approxOrder = 1
internalNodes = 0
quadElements = 1

1

In [744]:
# Geometria megrajzolása
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p33 = gmsh.model.occ.addPoint(base / 2, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
l3 = gmsh.model.occ.addLine(p3, p33)
l33 = gmsh.model.occ.addLine(p33, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l33, l4])

sf1 = gmsh.model.occ.addPlaneSurface([cl1])

gmsh.model.occ.synchronize()

In [745]:
# Megfogások és terhelések helyének megadása
# az egyes peremfeltételek helyét megadó geometriai objektumokat
# csoportokba rendezzük

# megfogás 1
supp1 = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1, supp1, "FixBottomY")

# megfogás 2
supp2 = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, supp2, "FixLeftX")

# terhelés
trac1 = gmsh.model.addPhysicalGroup(1, [l33])
gmsh.model.setPhysicalName(1, trac1, "ForceTopY")

In [746]:
# Végeselemes felosztás elvégzése

# lekérjük az összes csomópontot
all = gmsh.model.getEntities(0)
# megadjuk, hogy a csomóponthoz rendelt eleméret mekkora legyen
gmsh.model.mesh.setSize(all, elemSize)
# kiválasztjuk a 8-as számú hálózó algoritmust a 2D-s sf1 felülethez
gmsh.model.mesh.setAlgorithm(2, sf1, 8)
# legeneráljuk a hálót a felület kontúrjához (1D-s)
gmsh.model.mesh.generate(1)
# legeneráljuk a hálót a felülethez (2D-s)
gmsh.model.mesh.generate(2)
# a legenerált háromszög elemeket négyszög elemekké alakítjuk
if quadElements == 1
    gmsh.model.mesh.recombine()
end
# másodfokú elemekhez:
# belső csomópontok használata
if internalNodes == 1
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 0) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
else
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
end
# közelítés fokszáma (1-től 5-ig)
gmsh.model.mesh.setOrder(approxOrder)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.00104616s, CPU 0.00104s)
Info    : 800 nodes 805 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 4.03546s, CPU 3.98735s)
Info    : 40403 nodes 80809 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 119606 internal 796 closed
Info    : Blossom recombination completed (Wall 0.996265s, CPU 0.996019s): 40002 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.99976, min Q = 0.600025
Info    : Done recombining 2D mesh (Wall 1.00339s, CPU 1.00314s)


In [747]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [748]:
# Anyagállandók mátrixa SA feladathoz
E = 2e5
ν = 0.3
D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2]

3×3 Matrix{Float64}:
 2.69231e5  1.15385e5      0.0
 1.15385e5  2.69231e5      0.0
 0.0        0.0        76923.1

In [749]:
# Anyagállandók mátrixa ÁSF feladathoz
E = 2e5
ν = 0.3
D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Float64}:
     2.1978e5  65934.1           0.0
 65934.1           2.1978e5      0.0
     0.0           0.0       76923.1

In [750]:
a = [[1, 2, 3, 7], [4, 5, 6, 3, 5], [7, 8]]
sum([length(a[i]) for i in 1:length(a)])

11

In [751]:
# Merevségi mátrix felépítése
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
# a lefoglalandó memória méretének becslése
lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
nn = []
I = []
J = []
V = []
V = convert(Vector{Float64}, V)
sizehint!(I, lengthOfIJV)
sizehint!(J, lengthOfIJV)
sizehint!(V, lengthOfIJV)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2approxOrder))
    numIntPoints = length(intWeights)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numIntPoints)
    Iidx = zeros(Int, numNodes * 2, numNodes * 2)
    Jidx = zeros(Int, numNodes * 2, numNodes * 2)
    for k in 1:numNodes*2, l in 1:numNodes*2
        Iidx[k, l] = l
        Jidx[k, l] = k
    end
    ∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
    B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
    K1 = zeros(2 * numNodes, 2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        for k in 1:numIntPoints
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        ∂h *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        B *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        K1 *= 0
        for k in 1:numIntPoints
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            K1 += B1' * D * B1 * jacDet[k] * intWeights[k]
        end
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, K1[:])
    end
    push!(nn, nnet)
end
K = sparse(I, J, V)
K0 = copy(K)
display(K)

80806×80806 SparseMatrixCSC{Float64, Int64} with 1444876 stored entries:
⎡⣿⣿⣤⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⢿⣿⣿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠹⢿⣿⣿⣷⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠙⢿⣿⣿⣧⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠉⢻⣿⣿⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠙⠻⣿⣿⣶⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠿⣿⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⡄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⠆⎦

In [752]:
# Teher vektor felépítése
p = [0, -1]
dof, dof = size(K)
f = zeros(dof)
tags = gmsh.model.getEntitiesForPhysicalGroup(1, trac1)
for i ∈ 1:length(tags)
    elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(1, tags[i])
    if length(elementTypes) != 1
        error("A peremen nem csak egyfajta végeselem van!")
    end
    #elements = elementTags[1]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[1])
    nnoe = reshape(elemNodeTags[1], numNodes, :)'
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[1], "Gauss" * string(approxOrder))
    numIntPoints = length(intWeights)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "Lagrange")
    h = reshape(fun, :, numIntPoints)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    H = zeros(2 * numIntPoints, 2 * numNodes)
    for j in 1:numIntPoints
        for k in 1:numNodes
            H[j*2-1, k*2-1] = h[k, j]
            H[j*2-0, k*2-0] = h[k, j]
        end
    end
    f1 = zeros(2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for l in 1:length(elementTags[1])
        elem = elementTags[1][l]
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        f1 *= 0
        for j in 1:numIntPoints
            H1 = H[j*2-1:j*2, 1:2*numNodes]
            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2)
            f1 += H1' * p * Ja * intWeights[j]
        end
        nn2[1:2:2*numNodes] = 2 * nnoe[l, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnoe[l, 1:numNodes]
        f[nn2] += f1
    end
end
display(f)

80806-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
 -2.5
  0.0
 -2.5
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [753]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)

80806×80806 SparseMatrixCSC{Float64, Int64} with 1436396 stored entries:
⎡⣿⣿⣤⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⢿⣿⣿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠹⢿⣿⣿⣷⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠙⢿⣿⣿⣧⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠉⢻⣿⣿⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠙⠻⣿⣿⣶⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠿⣿⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⡄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⠆⎦

In [754]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
nodeTags .-= 1
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)

80806×80806 SparseMatrixCSC{Float64, Int64} with 1433599 stored entries:
⎡⣿⣿⣤⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⢿⣿⣿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠹⢿⣿⣿⣷⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠙⢿⣿⣿⣧⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠉⢻⣿⣿⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠙⠻⣿⣿⣶⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠿⣿⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣷⣆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⣿⣿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⡄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⠆⎦

In [755]:
q = K \ f

80806-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
 -0.0037780319866222143
 -0.002460558915635882
 -0.00406465544274078
 -0.005878021759506307
  0.0
  0.0
  ⋮
 -0.0012507656892409799
 -0.0008098197020409346
 -0.0012048470794414168
 -0.000863585423853363
 -0.0012448809169725774
 -0.000806713666426642
 -0.0011862874274737021
 -0.0008457778917184465
 -0.001232263548111023

In [756]:
p[2] / (E / height)

-0.005

In [757]:
-(p[2] / E) * ν * base

0.0015

In [758]:
K0 * q

80806-element Vector{Float64}:
  2.5532901737836022
  7.352128196419547
  0.13034876881310498
 -0.4134555438148235
 -1.0297201406309127e-13
 -2.499999999999943
 -2.842170943040401e-14
 -2.4999999999999716
  1.7992816817147927
  9.613237859469926
  ⋮
 -1.7186252421197423e-13
  4.263256414560601e-14
  1.0658141036401503e-14
  1.7763568394002505e-14
  3.410605131648481e-13
 -1.709743457922741e-14
 -5.684341886080802e-14
  9.947598300641403e-14
 -1.1368683772161603e-13

In [759]:
k = base * E / height

200000.0

In [760]:
# Feszültségek számítása
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
numElem = Int[]
σ = Vector{Float64}[]
σx = Vector{Float64}[]
σy = Vector{Float64}[]
σxy = Vector{Float64}[]
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    s0 = zeros(3numNodes) # csak SA és ÁSF feladatnál, FSZ-nél már 4 kell
    nodeCoord = zeros(numNodes * 3)
    for k in 1:dim, j = 1:numNodes
        nodeCoord[k+(j-1)*3] = localNodeCoord[k+(j-1)*dim]
    end
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, nodeCoord, "GradLagrange")
    ∇h = reshape(dfun, :, numNodes)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numNodes)
    ∂h = zeros(3, numNodes * numNodes)
    B = zeros(3 * numNodes, 2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, nodeCoord)
        Jac = reshape(jac, 3, :)
        for k in 1:numNodes
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        ∂h *= 0
        for k in 1:numNodes, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        B *= 0
        for k in 1:numNodes, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        push!(numElem, elem)
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        s = zeros(9numNodes) # tenzornak 9 eleme van
        sx = zeros(numNodes)
        sy = zeros(numNodes)
        sxy = zeros(numNodes)
        for k in 1:numNodes
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            s0 = D * B1 * q[nn2]
            s[(k-1)*9+1:k*9] = [s0[1], s0[3], 0, s0[3], s0[2], 0, 0, 0, 0]
            sx[k] = s0[1]
            sy[k] = s0[2]
            sxy[k] = s0[3]
        end
        push!(σ, s)
        push!(σx, sx)
        push!(σy, sy)
        push!(σxy, sxy)
    end
end

In [761]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)

In [762]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uvec = gmsh.view.add("uvec")
u = zeros(3 * non)
for i in 1:length(nodeTags)
    u[3i-2] = q[2*nodeTags[i]-1]
    u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uvec, 0, "rectangle", "NodeData", nodeTags, u, 0, 3)


In [763]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
ux = gmsh.view.add("ux")
u = zeros(non)
for i in 1:length(nodeTags)
    u[i] = q[2*nodeTags[i]-1]
    #u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(ux, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)


In [764]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uy = gmsh.view.add("uy")
u = zeros(non)
for i in 1:length(nodeTags)
    #u[3i-2] = q[2*nodeTags[i]-1]
    u[i] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uy, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)


In [765]:
S = gmsh.view.add("σ")
gmsh.view.addModelData(S, 0, "rectangle", "ElementNodeData", numElem, σ, 0, 9)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [766]:
Sx = gmsh.view.add("σx")
gmsh.view.addModelData(Sx, 0, "rectangle", "ElementNodeData", numElem, σx, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [767]:
Sy = gmsh.view.add("σy")
gmsh.view.addModelData(Sy, 0, "rectangle", "ElementNodeData", numElem, σy, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


5

In [768]:
Sxy = gmsh.view.add("τxy")
gmsh.view.addModelData(Sxy, 0, "rectangle", "ElementNodeData", numElem, σxy, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


6

In [769]:
#gmsh.write("rect.geo")
#gmsh.write("rect.msh")
#gmsh.write("rect.pos")

In [770]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [771]:
gmsh.finalize()